# Assignment 1 - SCARA robot with YZX spherical wrist 

## Robot diagram

![](SCARA_diagram.png)

## Forward kinematics

To solve forward kinematics we just use homogeneous transformation.  
![](Forward_kinematics.jpg)

## Inverse kinematics

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Solution is devided into two parts. At first we solve task for robot manipulator, then for robot wrist.  

### Robot manipulator part

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As we know position vector for wrist is equal to robot manipulator position. So for robot manupulator we can use geometric approach.

![](Robot_manipulator_inverse_kinematics.jpg)

### Robot wrist part

![](Wrist_inverse_kinematics.jpg)

## Code implementation

In [1]:
%matplotlib widget
import numpy as np
from numpy import pi
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
np.set_printoptions(suppress=True)

#This class is require for handy matrix multiplication.
class Matrix:
    
    __array_ufunc__ = None
    
    def __init__(self, val):
        self.val = val
        
    def __mul__(self, b):
        if isinstance(b, Matrix):
            val = np.matmul(self.val, b.val)
        else:
            val = np.matmul(self.val, b)
        return Matrix(val)
    
    def __rmul__(self, a):
        if isinstance(a, Matrix):
            val = np.matmul(a.val, self.val)
        else:
            val = np.matmul(a, self.val)
        return Matrix(val)

#Rotation around x axis
class Rx(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[0,0] = 1
        rot[1,:] = [0, np.cos(q), -np.sin(q),0]
        rot[2,:] = [0, np.sin(q), np.cos(q),0]
        self.val = rot

#Rotation around y axis
class Ry(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[:3,:3] = [[np.cos(q), 0, np.sin(q)],
         [0, 1, 0],
         [-np.sin(q), 0, np.cos(q)]
        ]
        self.val = rot

#Rotation around z axis
class Rz(Matrix):
    def __init__(self, q):
        rot = np.zeros((4,4),dtype=np.float64)
        rot[3,3] = 1
        rot[0,:] = [np.cos(q), -np.sin(q),0,0]
        rot[1,:] = [np.sin(q), np.cos(q),0,0]
        rot[2,2] = 1
        self.val = rot

#Transition matrix
class T(Matrix):
    def __init__(self, vector):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[0:3,3] = vector
        np.fill_diagonal(mat,1)  
        self.val = mat

#Transition matrix along x axis
class Tx(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[0,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

#Transition matrix along y axis
class Ty(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[1,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

#Transition matrix along z axis
class Tz(Matrix):
    def __init__(self, q):
        mat = np.zeros((4,4),dtype=np.float64)
        mat[2,3] = q
        np.fill_diagonal(mat,1)  
        self.val = mat

In [2]:
class ScaraYZX(widgets.HBox):
    
    def extract_plot_points(serial): 
        xs, ys, zs = [0],[0],[0]
        for trans in serial: 
            x,y,z = trans.val[0:3,3]
            xs.append(x)
            ys.append(y)
            zs.append(z)

        return xs,ys,zs


    def extract_vectors_from_trans(trans):
        x,y,z = trans[0:3,3]
        p = [x,y,z]
        v1 = trans[0:3,0]
        v2 = trans[0:3,1]
        v3 = trans[0:3,2]

        return p, [v1,v2,v3]

    def plot_arrow(ax, p,v,color):
        x,y,z = p 
        u,v,w = v 
        ax.quiver(x, y, z, u, v, w, length=0.5, normalize=True,color=color)
    
    def plot_frame(ax, trans): 
        position, rotations = ScaraYZX.extract_vectors_from_trans(trans.val)
        colors = ['r', 'g', 'b']
        for i in range(3): 
            ScaraYZX.plot_arrow(ax, position, rotations[i], colors[i])

            
    def update_q1(self, q1):
        self.R1 = Rz(q1.new)
        self.draw() 
 
    def update_q2(self, q2):
        self.R2 = Rz(q2.new)
        self.draw()

    def update_q4(self, q4):
        self.q4 = q4.new
        self.T35 = Tz(-self.L3-q4.new)
        self.draw()

    def update_q5(self, q5):
        self.R5 = Ry(q5.new)
        self.draw()
        
    def update_q6(self, q6):
        self.R6 = Rz(q6.new)
        self.draw()
        
    def update_q7(self, q7):
        self.R7 = Rx(q7.new)
        self.draw()
        
    def __init__(self):
        super().__init__()
        
        self.Lbase, self.L1, self.L2, self.L3 = 1, 1, 1, 1 
        p_tool = [1,1,1]
        q1, q2, q4, q5, q6, q7 = 0, 0, 0, 0, 0, 0
        
        self.Tbase = Tz(self.Lbase)
        self.R1 = Rz(q1)
        self.T12 = Tx(self.L1)
        self.R2 = Rz(q2)
        self.T23 = Tx(self.L2)
        self.T35 = Tz(-self.L3-q4)
        self.R5 = Ry(q5)
        self.R6 = Rz(q6)
        self.R7 = Rx(q7)
        self.T7_tool = T(p_tool)
        
        self.A_res =  self.Tbase * self.R1 * self.T12 * self.R2 * \
                    self.T23 * self.T35 * self.R5 * self.R6 * self.R7 * self.T7_tool

        output = widgets.Output()
        
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(6.5, 6.5))
            self.ax = self.fig.add_subplot(111, projection='3d')
        
        
        self.fig.canvas.toolbar_position = 'bottom'
        self.ax.grid(True)

        q1_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q1, description='q1')
        q2_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q2, description='q2')
        q4_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q4, description='q4')
        q5_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q5, description='q5')
        q6_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q6, description='q6')
        q7_slider = widgets.FloatSlider(min=0, max=2 * pi, step=0.01, value=q7, description='q7')
        
        q1_slider.observe(self.update_q1, 'value')
        q2_slider.observe(self.update_q2, 'value')
        q4_slider.observe(self.update_q4, 'value')
        q5_slider.observe(self.update_q5, 'value')
        q6_slider.observe(self.update_q6, 'value')
        q7_slider.observe(self.update_q7, 'value')
        
        self.A_res_widget = widgets.Textarea(value='', disabled=True, layout={'height': '300px'})
        
        controls = widgets.VBox([
            q1_slider, q2_slider, q4_slider, q5_slider, q6_slider, q7_slider,
            self.A_res_widget
            
        ])
        controls.layout = widgets.Layout(
            display='flex',
            flex='flex-shrink'
         )
        
        out_box = widgets.Box([output])
        output.layout = widgets.Layout(
            display='flex',
            flex='flex-shrink'
         )

        self.children = [controls, output]
        self.draw()
    
    
    def forward(self):
        A1 = self.Tbase * self.R1
        A2 = self.T12 * self.R2
        A3 = self.T23
        Awrist = self.T35 * self.R5 * self.R6 * self.R7  
        Atool = self.T7_tool
        
        A = [A1, A2, A3, Awrist, Atool]
        states = [Matrix(np.eye(4))]
        for a in A:
            states.append(states[-1] * a)
            
        self.A_res = states[-1]

        return states
    
    def inverse(self):
        A07 = self.A_res * np.linalg.inv(self.T7_tool.val)
        position, rotations = ScaraYZX.extract_vectors_from_trans(A07.val)
        px, py, pz = position
        
        cos_q2 = (px**2 + py**2 - (self.L1**2 + self.L2**2))/(2*self.L1*self.L2)
        cos_q2 = np.clip(cos_q2, -1.0, 1.0)
        sin_q2 = (1-cos_q2**2)**0.5
        
        alpha = np.arctan2(py, px)
        beta = np.arctan2(self.L2*sin_q2, self.L1 + self.L2*cos_q2)
        q1 = alpha - beta
        q2 = np.arctan2(sin_q2, cos_q2)
        q4 = self.Lbase - self.L3 - pz 

        A05 = self.Tbase * Rz(q1) * self.T12 * Rz(q2) * self.T23 * Tz(-self.L3-q4)
        R57 = np.linalg.inv(A05.val) * A07
        R57 = R57.val
        if R57[0][0]==0 and R57[2][0]==0:
            if R57[1][0] < 0:
                q6 = -np.pi/2
            else:
                q6 = np.pi/2
            q7 = 0
            q5 = np.arctan2(R57[0][2], R57[2][2])
        else:
            q5 = np.arctan2(-R57[2][0], R57[0][0])
            q7 = np.arctan2(-R57[1][2], R57[1][1])
            q6 = np.arctan2(np.cos(q7) * R57[1][0], R57[1][1])
        return q1,q2,q4,q5,q6,q7
        
        
    def draw(self):
        plt.cla()
        
        states = self.forward()
        q1,q2,q4,q5,q6,q7 = self.inverse()
        
        result_message = "Forward kinematics result matrix:\n" + np.array2string(self.A_res.val, precision=2) + "\n\n"
        result_message += "Inverse kinematics solution:\n"
        result_message += "q1: " + '{:.2f}'.format(q1) + "\n"
        result_message += "q2: " + '{:.2f}'.format(q2) + "\n"
        result_message += "q4: " + '{:.2f}'.format(q4) + "\n"
        result_message += "q5: " + '{:.2f}'.format(q5) + "\n"
        result_message += "q6: " + '{:.2f}'.format(q6) + "\n"
        result_message += "q7: " + '{:.2f}'.format(q7) + "\n"
        self.A_res_widget.value = result_message
        
        for state in states:
            ScaraYZX.plot_frame(self.ax, state)

        xs,ys,zs = ScaraYZX.extract_plot_points(states)
        self.ax.plot(xs,ys,zs, linewidth=2)      
        self.ax.set_xlim3d([-3,3])
        self.ax.set_ylim3d([-3,3])
        self.ax.set_zlim3d([-3,3])
        plt.show() 
        
ScaraYZX()

ScaraYZX(children=(VBox(children=(FloatSlider(value=0.0, description='q1', max=6.283185307179586, step=0.01), …